In [115]:
import json
import re
from mgrs_convert import mgrs_to_lat_long
from collections import defaultdict

In [116]:
fields = {'patient_precedence': {'a':'urgent','b':'urgent surgical', 'c':'priority','d':'routine','e':'convenience'},
          'equipment':{'a':'None','b':'hoist','c':'extraction equipment','d':'ventilator'},
          'mobility' : {'l':'litter','a':'ambulatory'},
          'site_security' : {'n':'No enemy troops in area', 'p':'Possible enemy troops in area', 'e':'Enemy troops in area - no escort',
                        'x':'Enemy troops in area - escort'},
          'pickup_mark':{'a':'Panels','b':'Pyrotechnic Signal','c': 'Smoke Signal','d':'None','e':'Other'},            
          'nationality' : {'a':'us military','b':'us civilian', 'c':'non us military',
                           'd':'non us civilian','e':'enemy prisoner of war'},
           'cbrn':{'c':'chemical','b':'biological','r':'radioactive','n':'nuclear'}
            }

line_num = {1:'location', 2:'freq', 3:'patient_precedence', 4:'equipment',
             5:'mobility', 6:'site_security', 7:'pickup_mark', 8:'nationality',
             9:'cbrn'}

In [117]:
with open('../data/nine_lines/nine_line_test.json') as f:
    nine_line_output = json.load(f)

In [118]:
with open('../data/lookup_tables/numbers.json') as f:
    number_dict = json.load(f)

with open('../data/lookup_tables/phonetic.json') as f:
    phonetic = json.load(f)    

In [119]:
nine_line_output

{'id': 'c3178739-2f4e-4e4d-a335-c1aa3ce733f4',
 'audio_path': '/Users/alex/Dev/hackathon/opmed/apinine_line_dom.wav',
 'audio_translation': 'nine line as follows line one three six uniform x-ray victor eight one four seven two four four five line two three niner three niner echo seven romeo line three one charlie two delta line four alpha line five one lima to alpha one line six november line seven alpha line eight three alpha over'}

In [120]:
nine_line_dict = {}

i=1


#find line one start
line_one_start = re.search('line one',nine_line_output.get('audio_translation'))
line_one_start = line_one_start.start()

for line in nine_line_output.get('audio_translation')[line_one_start:].split('line'):
    if line:
        nine_line_dict[line_num[i]] = ' '.join(line.split()[1:])
        i+=1

In [121]:
nine_line_dict

{'location': 'three six uniform x-ray victor eight one four seven two four four five',
 'freq': 'three niner three niner echo seven romeo',
 'patient_precedence': 'one charlie two delta',
 'equipment': 'alpha',
 'mobility': 'one lima to alpha one',
 'site_security': 'november',
 'pickup_mark': 'alpha',
 'nationality': 'three alpha over'}

In [122]:
def check_for_phonetic(word:str)->str:
   match = [item for item in phonetic.get('phonetic_alphabet') if item == word]
   return match[0][0] if match else ''

def check_for_number(word:str)->str:
   match = [item for item in list(number_dict.get('digits').keys()) if item == word]
   return  str(number_dict.get('digits')[match[0]]) if match else ''

def get_field(letter:str,field_name:str)->str:
   match = [v for k,v in fields.get(field_name).items() if k == letter]
   return match[0] if match else 'None'

In [123]:
formatted_nine_liner = {}
for k,v in nine_line_dict.items():
    output=''
    
    #get fields with associated values and process
    if k in ['patient_precedence','mobility','nationality']:
        for idx,word in enumerate(v.split()):
            result = check_for_phonetic(word)
            if result:
                field = get_field(result,k)
                try:
                    quantity = check_for_number(v.split()[idx-1])
                except:
                    quantity = check_for_number(v.split()[idx+1])
                formatted_nine_liner[field] = quantity

    else:
        for idx,word in enumerate(v.split()):
            result = check_for_phonetic(word)
            if result:
                output+=result
            else:
                output+=check_for_number(word)
        if k == 'location':
            lat,lon = mgrs_to_lat_long(output)
            formatted_nine_liner[k] = {'mgrs':output,'lat':lat,'lon':lon}
        elif k in ['equipment','site_security','pickup_mark']:
            field = get_field(result,k)
            formatted_nine_liner[k]=field
        else:
            formatted_nine_liner[k]=output


In [124]:
nine_line_dict

{'location': 'three six uniform x-ray victor eight one four seven two four four five',
 'freq': 'three niner three niner echo seven romeo',
 'patient_precedence': 'one charlie two delta',
 'equipment': 'alpha',
 'mobility': 'one lima to alpha one',
 'site_security': 'november',
 'pickup_mark': 'alpha',
 'nationality': 'three alpha over'}

In [125]:
formatted_nine_liner

{'location': {'mgrs': '36uxv81472445',
  'lat': 48.946327719344,
  'lon': 35.47848991303637},
 'freq': '3939e7r',
 'priority': '1',
 'routine': '2',
 'equipment': 'None',
 'litter': '1',
 'ambulatory': '2',
 'site_security': 'No enemy troops in area',
 'pickup_mark': 'Panels',
 'us military': '3'}

In [129]:
import nineline_entry

In [130]:
def send_to_db(nineline:dict):
    id_ = nine_line_output.get('id')
    audio_transcription = nine_line_output.get('audio_translation')
    audio_file = nine_line_output.get('audio_path')
    mgrs = nineline.get('location')['mgrs']
    lat =  nineline.get('location')['lat']
    lon = nineline.get('location')['lon']
    frequency = nineline.get('freq')
    urgent = nineline.get('urgent')
    urgent_surgical = nineline.get('urgent surgical')
    priority= nineline.get('priority')
    routine = nineline.get('routine')
    convenience = nineline.get('convenience')
    litter = nineline.get('litter')
    ambulatory = nineline.get('ambulatory')
    us_military = nineline.get('us military')
    us_civilian = nineline.get('us civilian')
    non_us_military = nineline.get('non us military')
    non_us_civilian = nineline.get('non us civilian')
    enemy_prisoner_of_war = nineline.get('enemy prisoner of war')
    equipment = nineline.get('equipment')
    site_security = nineline.get('site_security')
    pickup_mark = nineline.get('pickup_mark')
    cbrn = nineline.get('cbrn')

    nineline_entry.insert(id_,audio_transcription,audio_file,mgrs,lat,lon,frequency,
                          urgent,urgent_surgical,priority,routine,convenience,litter,
                          ambulatory, us_military, us_civilian, non_us_military,
                          non_us_civilian, enemy_prisoner_of_war,equipment,site_security,pickup_mark,cbrn)

In [131]:
send_to_db(formatted_nine_liner)

TypeError: insert() takes 22 positional arguments but 23 were given

In [69]:
def text2digits(textnum, numwords={}):
    if not numwords:
      units = [
        "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen",
      ]

      tens = ["", "", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"]

      scales = ["hundred", "thousand", "million", "billion", "trillion"]

      numwords["and"] = (1, 0)
      for idx, word in enumerate(units):    numwords[word] = (1, idx)
      for idx, word in enumerate(tens):     numwords[word] = (1, idx * 10)
      for idx, word in enumerate(scales):   numwords[word] = (10 ** (idx * 3 or 2), 0)

    current = 0
    result=''
    for word in textnum.split():
        if word not in numwords and len(word)>1:
            continue
        elif word not in numwords:
            result+=word
            continue
          #raise Exception("Illegal word: " + word)

        scale, increment = numwords[word]
        current = current * scale + increment
        result+=str(increment)
        if scale > 100:
            result += current
            current = 0
   # print(temp_str)
    return result



In [103]:
def locate_str(term: str, text_str: str):
     match_obj = re.search(term,text_str)
     if match_obj:
          return match_obj.end() 

In [117]:

def extract_str_values(option:str,option_list:list ,text_str:str)->str:
    item_list = text_str.split()
    for item in item_list:
        if item in option_list and item == option:
            return text2digits(item_list[item_list.index(item)+1])

In [118]:
nine_line_converted = {}
nine_line_converted['location'] = text2digits(nine_line_dict['location'])
nine_line_converted['freq'] = text2digits(nine_line_dict['freq'])
nine_line_converted['patient_precedence'] = {'status':nine_line_dict['patient_precedence'].split()[0],
                                             'num_patients':text2digits(nine_line_dict['patient_precedence'])} 
nine_line_converted['equipment'] = nine_line_dict['equipment']
nine_line_converted['number_patients'] = {'litter': 
                                          extract_str_values('litter', mobility, nine_line_dict.get('num_patients')),
                                          'ambulatory': 
                                          extract_str_values('ambulatory', mobility, nine_line_dict.get('num_patients'))}

nine_line_converted['site_security'] = nine_line_dict['site_security']
nine_line_converted['pickup_mark'] = nine_line_dict['pickup_mark']
nine_line_converted['nationality'] = nine_line_dict['nationality']
nine_line_converted['cbrn'] = nine_line_dict['cbrn']



In [120]:
def check_field_options(field_list: list, item:dict):
    for field in field_list:
        m = re.search(field,item)
        return field if m else False

In [124]:
print(check_field_options(status,nine_line_dict.get('patient_precedence')))    

urgent


In [126]:
nine_line_converted

{'location': '39sxs2983701890',
 'freq': '125',
 'patient_precedence': {'status': 'urgent', 'num_patients': '3'},
 'equipment': 'ventilator',
 'number_patients': {'litter': '1', 'ambulatory': '2'},
 'site_security': 'no enemies in area',
 'pickup_mark': 'smoke signal',
 'nationality': 'US military three',
 'cbrn': 'chemical'}

In [ ]:
nine_line_object = {''}

In [5]:
sample_nine_line

'line one thirty nine s x s two nine eight three seven zero one eight nine zero line two frequency one two five line three '